In [1]:
# Utils packages
import ast
from pathlib import Path, PurePath
from typing import Union

# Data collector packages
from hentai import Utils, Hentai, Option

# Analysis packages
import pandas as pd
from PIL import Image
from tqdm import tqdm

# Compute packages
import numpy as np
import tensorflow as tf

## Data collection

We will use the information and media collected on the nhentai.net website. Recording each entry in a 'metadata.csv' file that aggregates information about the hentai, including links to download pages.

In [2]:
N_SAMPLE = 1 # Number of sample to refetch from hentai.
DATA_PATH = "data" # Directory to drop collected data.

In [3]:
def random_hentai(_: int):
    # Request a random hentai
    hentai = Utils.get_random_hentai()
    # A hentai will be stored based on the gallery ID instead of the medai ID
    hentai_path = Path(DATA_PATH) / "raw" / str(hentai.id)
    # Since downloading a hentai is an expensive operation, we only
    # download when the hentai ID directory does not exist
    if not hentai_path.is_dir():
        hentai = Hentai(hentai.id)
        hentai.download(folder=hentai_path, progressbar=True)
    return pd.Series(hentai.dictionary(Option.all()))

In [4]:
# Query the nhentai.net API to get N number of random hentai
collected_hentais_df = pd.Series(range(N_SAMPLE), dtype="Int64")
collected_hentais_df = collected_hentais_df.apply(random_hentai)

Download #337224: 100%|○○○○○○○○○○| 42/42 [00:09<00:00,  4.30page/s]


## Read dataset file

In [5]:
# The metadata NDJSON file that will contain the raw information about the hentai
metadata_path = Path(DATA_PATH) / "raw" / "metadata.ndjson"

In [6]:
hentais_df = pd.read_json(
    metadata_path,
    orient="records",
    lines=True
)
hentais_df

,id,title,scanlator,url,api,media_id,epos,num_favorites,tag,group,parody,character,language,artist,category,cover,thumbnail,image_urls,num_pages,0
0,352167,Boku wa Chiisana Succubus no Shimobe - I am a ...,None,https://nhentai.net/g/352167,https://nhentai.net/api/gallery/352167,1872914,1616460447,20119,"[wings, garter belt, group, uncensored, story ...",[],[],[],"['english', 'translated']",[tamano kedama],[manga],https://t.nhentai.net/galleries/1872914/cover.jpg,https://t.nhentai.net/galleries/1872914/thumb.jpg,[https://i.nhentai.net/galleries/1872914/1.jpg...,195,1.0
1,212548,Syoku,None,https://nhentai.net/g/212548,https://nhentai.net/api/gallery/212548,1133892,1509103975,4452,"[big breasts, milf, nakadashi, sole female, ch...",[hyoco road],[occultic nine],[ririka nishizono],"['english', 'translated']",[hyocorou],[doujinshi],https://t.nhentai.net/galleries/1133892/cover.jpg,https://t.nhentai.net/galleries/1133892/thumb.jpg,[https://i.nhentai.net/galleries/1133892/1.jpg...,33,NaN
2,57170,Drug Fumi-chan Seishun Hen Ura Manga Michi,None,https://nhentai.net/g/57170,https://nhentai.net/api/gallery/57170,333208,1403992737,21,"[bondage, yuri, schoolgirl uniform, tankoubon]",[],[],[],['japanese'],[shin tsuguru],[manga],https://t.nhentai.net/galleries/333208/cover.jpg,https://t.nhentai.net/galleries/333208/thumb.jpg,"[https://i.nhentai.net/galleries/333208/1.jpg,...",166,NaN
3,75495,SHIRONA HAZARD,None,https://nhentai.net/g/75495,https://nhentai.net/api/gallery/75495,490119,1403996013,191,[big breasts],[draw go],[pokemon],[cynthia],['japanese'],[souichi],[doujinshi],https://t.nhentai.net/galleries/490119/cover.jpg,https://t.nhentai.net/galleries/490119/thumb.jpg,"[https://i.nhentai.net/galleries/490119/1.jpg,...",23,NaN
4,229284,2205-nen no Afterglow,None,https://nhentai.net/g/229284,https://nhentai.net/api/gallery/229284,1206999,1522895460,101,"[eyepatch, sole male, sole female]",[frozen world],[touken ranbu],[shokudaikiri mitsutada],['japanese'],[kanaso],[doujinshi],https://t.nhentai.net/galleries/1206999/cover.jpg,https://t.nhentai.net/galleries/1206999/thumb.jpg,[https://i.nhentai.net/galleries/1206999/1.jpg...,47,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3332,80905,The Report of Monster Girls 01-05,None,https://nhentai.net/g/80905,https://nhentai.net/api/gallery/80905,528410,1403996926,70,"[big breasts, monster girl, lolicon, paizuri]",[],[],[],['japanese'],[okayado],[manga],https://t.nhentai.net/galleries/528410/cover.png,https://t.nhentai.net/galleries/528410/thumb.png,"[https://i.nhentai.net/galleries/528410/1.png,...",8,NaN
3333,314463,Altrias true LOVE,None,https://nhentai.net/g/314463,https://nhentai.net/api/gallery/314463,1643885,1590284036,5303,"[armpit licking, big penis, sole male, anal, b...",[jikomankitsu.],[fate grand order],"[artoria pendragon, artoria pendragon alter, g...","['english', 'translated']",[wtwinmk2nd],[doujinshi],https://t.nhentai.net/galleries/1643885/cover.jpg,https://t.nhentai.net/galleries/1643885/thumb.jpg,[https://i.nhentai.net/galleries/1643885/1.jpg...,24,NaN
3334,19145,Fuuka Meiga za Nihon date: Roommate no Hime go...,None,https://nhentai.net/g/19145,https://nhentai.net/api/gallery/19145,68021,1403984982,17,"[lolicon, yuri]",[bolze.],[mai-otome],"[shizuru fujino, erstin ho, nina wang, yohko h...",['japanese'],[rit.],[doujinshi],https://t.nhentai.net/galleries/68021/cover.jpg,https://t.nhentai.net/galleries/68021/thumb.jpg,"[https://i.nhentai.net/galleries/68021/1.jpg, ...",18,NaN
3335,36054,Nan Demo-R Kepuuroku,,https://nhentai.net/g/36054,https://nhentai.net/api/gallery/36054,161287,1403988766,155,"[big breasts, bestiality, anal, bondage, demon...",[tsurikichi doumei],"[akihabara dennou gumi, gaogaigar, king of fig...","[morrigan aensland, yuri sakazaki, sakura kino...",[japanese],"[kogawa masayoshi, momoya show-neko, miwa tama...",[doujinshi],https://t.nhentai.net/galleries/161287/cover.jpg,https://t.nhentai.net/galleries/161287/thumb.jpg,"[https

## Save collected dataset

In [7]:
hentais_df = pd.concat([hentais_df, collected_hentais_df])
hentais_df = hentais_df.drop_duplicates("id")
hentais_df.to_json(
    metadata_path,
    orient="records",
    lines=True
)

## Remove corrupted images

In [8]:
def hentai_filenames(hentai: pd.Series):
    return [
        str(PurePath(str(hentai["id"])) / Path(url).name)
        for url in hentai["image_urls"]
    ]

hentais_df = hentais_df.assign(filenames=hentais_df.apply(hentai_filenames, axis=1))
hentais_df

,id,title,scanlator,url,api,media_id,epos,num_favorites,tag,group,...,character,language,artist,category,cover,thumbnail,image_urls,num_pages,0,filenames
0,352167,Boku wa Chiisana Succubus no Shimobe - I am a ...,None,https://nhentai.net/g/352167,https://nhentai.net/api/gallery/352167,1872914,1616460447,20119,"[wings, garter belt, group, uncensored, story ...",[],...,[],"['english', 'translated']",[tamano kedama],[manga],https://t.nhentai.net/galleries/1872914/cover.jpg,https://t.nhentai.net/galleries/1872914/thumb.jpg,[https://i.nhentai.net/galleries/1872914/1.jpg...,195,1.0,"[352167\1.jpg, 352167\2.jpg, 352167\3.jpg, 352..."
1,212548,Syoku,None,https://nhentai.net/g/212548,https://nhentai.net/api/gallery/212548,1133892,1509103975,4452,"[big breasts, milf, nakadashi, sole female, ch...",[hyoco road],...,[ririka nishizono],"['english', 'translated']",[hyocorou],[doujinshi],https://t.nhentai.net/galleries/1133892/cover.jpg,https://t.nhentai.net/galleries/1133892/thumb.jpg,[https://i.nhentai.net/galleries/1133892/1.jpg...,33,NaN,"[212548\1.jpg, 212548\2.jpg, 212548\3.jpg, 212..."
2,57170,Drug Fumi-chan Seishun Hen Ura Manga Michi,None,https://nhentai.net/g/57170,https://nhentai.net/api/gallery/57170,333208,1403992737,21,"[bondage, yuri, schoolgirl uniform, tankoubon]",[],...,[],['japanese'],[shin tsuguru],[manga],https://t.nhentai.net/galleries/333208/cover.jpg,https://t.nhentai.net/galleries/333208/thumb.jpg,"[https://i.nhentai.net/galleries/333208/1.jpg,...",166,NaN,"[57170\1.jpg, 57170\2.jpg, 57170\3.jpg, 57170\..."
3,75495,SHIRONA HAZARD,None,https://nhentai.net/g/75495,https://nhentai.net/api/gallery/75495,490119,1403996013,191,[big breasts],[draw go],...,[cynthia],['japanese'],[souichi],[doujinshi],https://t.nhentai.net/galleries/490119/cover.jpg,https://t.nhentai.net/galleries/490119/thumb.jpg,"[https://i.nhentai.net/galleries/490119/1.jpg,...",23,NaN,"[75495\1.jpg, 75495\2.jpg, 75495\3.jpg, 75495\..."
4,229284,2205-nen no Afterglow,None,https://nhentai.net/g/229284,https://nhentai.net/api/gallery/229284,1206999,1522895460,101,"[eyepatch, sole male, sole female]",[frozen world],...,[shokudaikiri mitsutada],['japanese'],[kanaso],[doujinshi],https://t.nhentai.net/galleries/1206999/cover.jpg,https://t.nhentai.net/galleries/1206999/thumb.jpg,[https://i.nhentai.net/galleries/1206999/1.jpg...,47,NaN,"[229284\1.jpg, 229284\2.png, 229284\3.jpg, 229..."
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3333,314463,Altrias true LOVE,None,https://nhentai.net/g/314463,https://nhentai.net/api/gallery/314463,1643885,1590284036,5303,"[armpit licking, big penis, sole male, anal, b...",[jikomankitsu.],...,"[artoria pendragon, artoria pendragon alter, g...","['english', 'translated']",[wtwinmk2nd],[doujinshi],https://t.nhentai.net/galleries/1643885/cover.jpg,https://t.nhentai.net/galleries/1643885/thumb.jpg,[https://i.nhentai.net/galleries/1643885/1.jpg...,24,NaN,"[314463\1.jpg, 314463\2.jpg, 314463\3.jpg, 314..."
3334,19145,Fuuka Meiga za Nihon date: Roommate no Hime go...,None,https://nhentai.net/g/19145,https://nhentai.net/api/gallery/19145,68021,1403984982,17,"[lolicon, yuri]",[bolze.],...,"[shizuru fujino, erstin ho, nina wang, yohko h...",['japanese'],[rit.],[doujinshi],https://t.nhentai.net/galleries/68021/cover.jpg,https://t.nhentai.net/galleries/68021/thumb.jpg,"[https://i.nhentai.net/galleries/68021/1.jpg, ...",18,NaN,"[19145\1.jpg, 19145\2.jpg, 19145\3.jpg, 19145\..."
3335,36054,Nan Demo-R Kepuuroku,,https://nhentai.net/g/36054,https://nhentai.net/api/gallery/36054,161287,1403988766,155,"[big breasts, bestiality, anal, bondage, demon...",[tsurikichi doumei],...,"[morrigan aensland, yuri sakazaki, sakura kino...",[japanese],"[kogawa masayoshi, momoya show-neko, miwa tama...",[doujinshi],https://t.nhentai.net/galleries/161287/cover.jpg,https://t.nhentai.net/galleries/161287/thumb.jpg,"[https://i.nhentai.net/galleries/161287/1.jpg,...",220,NaN,"[36054\1.jpg, 36054\2.jpg, 36054\3.jpg, 36054\..."

In [42]:
def preprocess_image(filename: str):
    image_path = Path(filename)
    image_preproc_dir = Path(DATA_PATH) / "preprocessed" / image_path.parent
    image_preproc_path = image_preproc_dir / Path(image_path.stem + ".jpg")
    # Preprocess only those images that do not exist in
    # the destination dataset for performance reason
    if not image_preproc_path.is_file():
        try:
            image = Image.open(Path(DATA_PATH) / "raw" / image_path)
            # Normalize images to be JPEG compatible
            image = image.convert("RGB")
            image_preproc_dir.mkdir(parents=True, exist_ok=True)
            image.save(image_preproc_path, "JPEG")
        except:
            return None

    return str(PurePath(image_preproc_path))

def preprocess_images(hentai: pd.Series):
    return [
        preprocess_image(filename)
        for filename in hentai["filenames"]
    ]

tqdm.pandas()
hentais_df = hentais_df.assign(filenames=hentais_df.progress_apply(preprocess_images, axis=1))

100%|██████████| 3337/3337 [00:16<00:00, 197.44it/s]


We will filter every hentais that contain at least one corrupted images, since pages have order dependencies.

In [43]:
hentais_df = hentais_df[hentais_df["filenames"].apply(all)]
hentais_df

,id,title,scanlator,url,api,media_id,epos,num_favorites,tag,group,...,character,language,artist,category,cover,thumbnail,image_urls,num_pages,0,filenames
0,352167,Boku wa Chiisana Succubus no Shimobe - I am a ...,None,https://nhentai.net/g/352167,https://nhentai.net/api/gallery/352167,1872914,1616460447,20119,"[wings, garter belt, group, uncensored, story ...",[],...,[],"['english', 'translated']",[tamano kedama],[manga],https://t.nhentai.net/galleries/1872914/cover.jpg,https://t.nhentai.net/galleries/1872914/thumb.jpg,[https://i.nhentai.net/galleries/1872914/1.jpg...,195,1.0,"[data\preprocessed\352167\1.jpg, data\preproce..."
1,212548,Syoku,None,https://nhentai.net/g/212548,https://nhentai.net/api/gallery/212548,1133892,1509103975,4452,"[big breasts, milf, nakadashi, sole female, ch...",[hyoco road],...,[ririka nishizono],"['english', 'translated']",[hyocorou],[doujinshi],https://t.nhentai.net/galleries/1133892/cover.jpg,https://t.nhentai.net/galleries/1133892/thumb.jpg,[https://i.nhentai.net/galleries/1133892/1.jpg...,33,NaN,"[data\preprocessed\212548\1.jpg, data\preproce..."
2,57170,Drug Fumi-chan Seishun Hen Ura Manga Michi,None,https://nhentai.net/g/57170,https://nhentai.net/api/gallery/57170,333208,1403992737,21,"[bondage, yuri, schoolgirl uniform, tankoubon]",[],...,[],['japanese'],[shin tsuguru],[manga],https://t.nhentai.net/galleries/333208/cover.jpg,https://t.nhentai.net/galleries/333208/thumb.jpg,"[https://i.nhentai.net/galleries/333208/1.jpg,...",166,NaN,"[data\preprocessed\57170\1.jpg, data\preproces..."
3,75495,SHIRONA HAZARD,None,https://nhentai.net/g/75495,https://nhentai.net/api/gallery/75495,490119,1403996013,191,[big breasts],[draw go],...,[cynthia],['japanese'],[souichi],[doujinshi],https://t.nhentai.net/galleries/490119/cover.jpg,https://t.nhentai.net/galleries/490119/thumb.jpg,"[https://i.nhentai.net/galleries/490119/1.jpg,...",23,NaN,"[data\preprocessed\75495\1.jpg, data\preproces..."
4,229284,2205-nen no Afterglow,None,https://nhentai.net/g/229284,https://nhentai.net/api/gallery/229284,1206999,1522895460,101,"[eyepatch, sole male, sole female]",[frozen world],...,[shokudaikiri mitsutada],['japanese'],[kanaso],[doujinshi],https://t.nhentai.net/galleries/1206999/cover.jpg,https://t.nhentai.net/galleries/1206999/thumb.jpg,[https://i.nhentai.net/galleries/1206999/1.jpg...,47,NaN,"[data\preprocessed\229284\1.jpg, data\preproce..."
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3332,80905,The Report of Monster Girls 01-05,None,https://nhentai.net/g/80905,https://nhentai.net/api/gallery/80905,528410,1403996926,70,"[big breasts, monster girl, lolicon, paizuri]",[],...,[],['japanese'],[okayado],[manga],https://t.nhentai.net/galleries/528410/cover.png,https://t.nhentai.net/galleries/528410/thumb.png,"[https://i.nhentai.net/galleries/528410/1.png,...",8,NaN,"[data\preprocessed\80905\1.jpg, data\preproces..."
3333,314463,Altrias true LOVE,None,https://nhentai.net/g/314463,https://nhentai.net/api/gallery/314463,1643885,1590284036,5303,"[armpit licking, big penis, sole male, anal, b...",[jikomankitsu.],...,"[artoria pendragon, artoria pendragon alter, g...","['english', 'translated']",[wtwinmk2nd],[doujinshi],https://t.nhentai.net/galleries/1643885/cover.jpg,https://t.nhentai.net/galleries/1643885/thumb.jpg,[https://i.nhentai.net/galleries/1643885/1.jpg...,24,NaN,"[data\preprocessed\314463\1.jpg, data\preproce..."
3334,19145,Fuuka Meiga za Nihon date: Roommate no Hime go...,None,https://nhentai.net/g/19145,https://nhentai.net/api/gallery/19145,68021,1403984982,17,"[lolicon, yuri]",[bolze.],...,"[shizuru fujino, erstin ho, nina wang, yohko h...",['japanese'],[rit.],[doujinshi],https://t.nhentai.net/galleries/68021/cover.jpg,https://t.nhentai.net/galleries/68021/thumb.jpg,"[https://i.nhentai.net/galleries/68021/1.jpg, ...",18,NaN,"[data\preprocessed\19145\1.jpg, data\preproces..."
3335,36054,Nan Demo-R Kepuuroku,,https://nhentai.net/g/36054,https://nhentai.net/api/gallery/36054,

In [44]:
# The metadata CSV file that will contain the raw information about the hentai
metadata_path = Path(DATA_PATH) / "preprocessed" / "metadata.ndjson"
hentais_df = hentais_df[[
    "id",
    "title",
    "num_favorites",
    "tag",
    "group",
    "parody",
    "character",
    "language",
    "artist",
    "category",
    "num_pages",
    "filenames"
]]
hentais_df.to_json(
    metadata_path,
    orient="records",
    lines=True
)
print("Number of preprocessed images :", len(hentais_df))

Number of preprocessed images : 3331
